# Model Training

Model training step for the project '**Aftershock pattern prediction based on earthquake rupture data for improved seismic hazard assessment**' (pred_seism_aftXYZ). DeVries18 will refer to the article 'Deep learning of aftershock patterns following large earthquakes' by Phoebe M. R. DeVries, Fernanda Viégas, Martin Wattenberg & Brendan J. Meade, and published in Nature in 2018 (https://www.nature.com/articles/s41586-018-0438-y ).

Inputs from previous steps of the process model:
-  'Features.pkl': Features and target of baseline model (DeVries18), as pickle file;
-  'new_features.pkl': New features and same target as baseline, as pickle file;
-  'model_baseline_prev.h5': Baseline deep neural net of DeVries18, as (untrained) Keras model HDF5 file;
-  'model_baseline_DeVries18_simplified_init.h5': Simplified topology for baseline DNN of DeVries18, as (untrained) Keras model HDF5 file;
-  'model_DNN_init.h5': Deep neural net topology proposed for this project, as (untrained) Keras model HDF5 file;
-  'model_ANN_init.h5': Shallow artificial neural net proposed for this project, as (untrained) Keras model HDF5 file.

For comparison with the DeVries study, we will use the same training and test sets ('Training_FileNames.h5', 'Testing_FileNames.h5'), first imported from the [Google Drive](https://drive.google.com/drive/folders/1c5Rb_6EsuP2XedDjg37bFDyf8AadtGDa)

## Split dataset into training and testing sets

In [1]:
import h5py
import numpy as np
import pandas as pd
import sklearn

h5file1 = h5py.File('./Data/Training_FileNames.h5', 'r')
training_filenames = np.array(h5file1.get('file_names_training'))
h5file2 = h5py.File('./Data/Testing_FileNames.h5', 'r')
testing_filenames = np.array(h5file2.get('file_names_testing'))

training_IDs_temp = map(lambda x: str(x, 'utf-8'), training_filenames)  
training_IDs = list(map(lambda x: 's' + x[0:x.find('_')], training_IDs_temp))
testing_IDs_temp = map(lambda x: str(x, 'utf-8'), testing_filenames)   
testing_IDs = list(map(lambda x: 's' + x[0:x.find('_')], testing_IDs_temp))

In [4]:
Features = pd.read_pickle('./Data/Features.pkl')
new_features = pd.read_pickle('./Data/new_features.pkl')

In [8]:
# create training & testing sets
TrainingSet_prev = Features.loc[Features['ID'].isin(training_IDs)]
TestingSet_prev = Features.loc[Features['ID'].isin(testing_IDs)]
TrainingSet_new = new_features.loc[new_features['ID'].isin(training_IDs)]
TestingSet_new = new_features.loc[new_features['ID'].isin(testing_IDs)]

# ratio of training samples
[len(TrainingSet_prev), len(TestingSet_prev), len(TrainingSet_prev)/(len(TrainingSet_prev)+len(TestingSet_prev))]

[164177, 38467, 0.8101744931998973]

In [9]:
# shuffle training sets
TrainingSet_prev = sklearn.utils.shuffle(TrainingSet_prev)
TrainingSet_new = sklearn.utils.shuffle(TrainingSet_new)

## Train baseline model

In [10]:
TrainingSet_prev.head(10)

,ID,aftershocksyn,posabsxx,posabsxy,posabsyy,posabsxz,posabsyz,posabszz,negabsxx,negabsxy,negabsyy,negabsxz,negabsyz,negabszz
28144,s1968TOKACH01NAGA,0.0,0.054881,0.024984,0.010592,0.007663,0.003465,0.009011,-0.054881,-0.024984,-0.010592,-0.007663,-0.003465,-0.009011
93258,s2004SUMATR01AMMO,1.0,0.019217,0.091083,0.202546,0.072640,0.021217,0.023352,-0.019217,-0.091083,-0.202546,-0.072640,-0.021217,-0.023352
72370,s2011TOHOKU01SATA,1.0,0.339550,0.334037,0.813508,0.514822,0.051104,0.581820,-0.339550,-0.334037,-0.813508,-0.514822,-0.051104,-0.581820
69997,s2010MAULEC01HAYE,1.0,1.847396,0.205944,1.139442,0.498965,0.613154,0.200377,-1.847396,-0.205944,-1.139442,-0.498965,-0.613154,-0.200377
72234,s2011TOHOKU01SATA,0.0,0.596709,0.283883,0.231554,0.300852,0.002066,0.034331,-0.596709,-0.283883,-0.231554,-0.300852,-0.002066,-0.034331
53030,s2004SUMATR02RHIE,1.0,1.177236,0.324688,0.544993,0.163446,0.074585,0.007162,-1.177236,-0.324688,-0.544993,-0.163446,-0.074585,-0.007162
28253,s2011TOHOKU02FUJI,1.0,1.175265,0.007689,0.631467,0.891350,0.668571,0.618322,-1.175265,-0.007689,-0.631467,-0.891350,-0.668571,-0.618322
161988,s2004SUMATR02RHIE,0.0,0.557100,3.431447,0.638794,0.068985,2.289272,2.090269,-0.557100,-3.431447,-0.638794,-0.068985,-2.289272,-2.090269
49494,s2011TOHOKU01SATA,1.0,0.505163,0.412470,0.305224,0.126985,0.099218,0.026316,-0.505163,-0.412470,-0.305224,-0.126985,-0.099218,-0.026316
58651,s2011TOHOKU01SATA,1.0,0.073595,0.063582,0.033686,0.288990,0.082774,0.055018,-0.073595,-0.063582,-0.033686,-0.288990,-0.082774,-0.055018


In [12]:
features = ['posabsxx', 'posabsxy', 'posabsyy', 'posabsxz', 'posabsyz', 'posabszz',
           'negabsxx', 'negabsxy', 'negabsyy', 'negabsxz', 'negabsyz', 'negabszz']

target = 'aftershocksyn'

x_train = TrainingSet_prev[features]
y_train = TrainingSet_prev[target]
x_test = TestingSet_prev[features]
y_test = TestingSet_prev[target]

x_test.to_pickle("./Data/TestingSet_X_prev.pkl")
y_test.to_pickle("./Data/TestingSet_y_prev.pkl")

In [14]:
import keras
from keras.models import load_model

# same as in DeVries18
baselinemodel_prev = load_model('./Models/model_baseline_prev.h5')
batch_size = 3500
epochs = 5

history = baselinemodel_prev.fit(x_train, y_train,
                            batch_size = batch_size,
                            epochs = epochs,
                            validation_split = 0.1)

Epoch 1/5
43/43 [==============================] - 4s 37ms/step - loss: 0.7467 - binary_accuracy: 0.5055 - val_loss: 0.7329 - val_binary_accuracy: 0.4976
Epoch 2/5
43/43 [==============================] - 1s 31ms/step - loss: 0.7439 - binary_accuracy: 0.5113 - val_loss: 0.7316 - val_binary_accuracy: 0.4991
Epoch 3/5
43/43 [==============================] - 1s 30ms/step - loss: 0.7452 - binary_accuracy: 0.5129 - val_loss: 0.7303 - val_binary_accuracy: 0.4999
Epoch 4/5
43/43 [==============================] - 1s 34ms/step - loss: 0.7445 - binary_accuracy: 0.5162 - val_loss: 0.7289 - val_binary_accuracy: 0.5013
Epoch 5/5
43/43 [==============================] - 2s 36ms/step - loss: 0.7432 - binary_accuracy: 0.5169 - val_loss: 0.7276 - val_binary_accuracy: 0.5021


This is the baseline model CV results - we should aim at achieving similar or greater accuracy ($\geq$ 72%) for the new model with new set of features. This will however not assure a same or better generalization than the baseline model (AUC = 85%, see next step of the process model).

In [16]:
baselinemodel_prev.save('./Models/model_baseline_prev_trained.h5')

### Side note

Since the Devries18 model uses a relatively small input layer (12 nodes), a simpler DNN topology should do just fine. We show it, only for illustrative purpose. The model proposed in this project will use another set of features.

In [18]:
baselinemodel_DeVries18_simplified = load_model('./Models/model_baseline_DeVries18_simplified_init.h5')

history = baselinemodel_DeVries18_simplified.fit(x_train, y_train,
                            batch_size = batch_size,
                            epochs = epochs,
                            validation_split = 0.1)

Epoch 1/5
43/43 [==============================] - 1s 9ms/step - loss: 0.6694 - binary_accuracy: 0.5168 - val_loss: 0.6645 - val_binary_accuracy: 0.5332
Epoch 2/5
43/43 [==============================] - 0s 6ms/step - loss: 0.6692 - binary_accuracy: 0.5266 - val_loss: 0.6644 - val_binary_accuracy: 0.5384
Epoch 3/5
43/43 [==============================] - 0s 7ms/step - loss: 0.6689 - binary_accuracy: 0.5330 - val_loss: 0.6643 - val_binary_accuracy: 0.5429
Epoch 4/5
43/43 [==============================] - 0s 6ms/step - loss: 0.6691 - binary_accuracy: 0.5369 - val_loss: 0.6642 - val_binary_accuracy: 0.5471
Epoch 5/5
43/43 [==============================] - 0s 5ms/step - loss: 0.6689 - binary_accuracy: 0.5425 - val_loss: 0.6641 - val_binary_accuracy: 0.5539


In [12]:
# note the similar CV accuracy using 12 - 8 - 8 - 1 instead of 12 - 50 - 50 - 50 - 50 - 50 - 50 - 1

In [19]:
baselinemodel_DeVries18_simplified.save('./Models/model_baseline_DeVries18_simplified_trained.h5')

## Train new models

Training new models with the proposed set of features based on geometry and kinematics (minimum distance to mainshock rupture and mean slip on rupture).

In [20]:
#features = ['mindist', 'dipMean', 'strikeMean', 'slipMean']   #best 4 - 12 - 12 - 1
features = ['mindist', 'slipMean']   #best 2 - 6 - 6 - 1
target = 'aftershocksyn'

x_train = TrainingSet_new[features]
y_train = TrainingSet_new[target]
x_test = TestingSet_new[features]
y_test = TestingSet_new[target]

# save for next step of the process model
x_test.to_pickle("./Data/TestingSet_X_new.pkl")
y_test.to_pickle("./Data/TestingSet_y_new.pkl")

### Train new DNN (simplified topology)

In [22]:
model_DNN = load_model('./Models/model_DNN_init.h5')

history = model_DNN.fit(x_train, y_train,
                            batch_size = batch_size,
                            epochs = epochs,
                            validation_split = 0.1)

Epoch 1/5
18/18 [==============================] - 1s 15ms/step - loss: 0.6839 - binary_accuracy: 0.6402 - val_loss: 0.6831 - val_binary_accuracy: 0.7086
Epoch 2/5
18/18 [==============================] - 0s 7ms/step - loss: 0.6840 - binary_accuracy: 0.6432 - val_loss: 0.6830 - val_binary_accuracy: 0.7086
Epoch 3/5
18/18 [==============================] - 0s 6ms/step - loss: 0.6839 - binary_accuracy: 0.6441 - val_loss: 0.6830 - val_binary_accuracy: 0.7086
Epoch 4/5
18/18 [==============================] - 0s 6ms/step - loss: 0.6839 - binary_accuracy: 0.6448 - val_loss: 0.6830 - val_binary_accuracy: 0.7086
Epoch 5/5
18/18 [==============================] - 0s 6ms/step - loss: 0.6837 - binary_accuracy: 0.6443 - val_loss: 0.6830 - val_binary_accuracy: 0.7086


In [ ]:
# Documenting a first DNN model result
# we tried the following topology 4 - 12 - 12 - 1, the 4 features being: ['mindist', 'dipMean', 'strikeMean', 'slipMean']
# we obtained: 
#Epoch 5/5
#147801/147801 [==============================] - 0s 2us/step - loss: 0.6134 - binary_accuracy: 0.6829 - 
#                                                           val_loss: 0.5801 - val_binary_accuracy: 0.7228

# additional tuning involved changes in topology, using relu instead of tanh activation, dropout rate change [NOT SHOWN for clarity].

In [23]:
model_DNN.save('./Models/model_DNN_trained.h5')

### Train ANN

Training of an Artificial Neural Network with one hidden layer (no deep learning!).

In [24]:
model_ANN = load_model('./Models/model_ANN_init.h5')

history = model_ANN.fit(x_train, y_train,
                            batch_size = batch_size,
                            epochs = epochs,
                            validation_split = 0.1)

Epoch 1/5
18/18 [==============================] - 1s 17ms/step - loss: 0.7372 - binary_accuracy: 0.2931 - val_loss: 0.7363 - val_binary_accuracy: 0.3011
Epoch 2/5
18/18 [==============================] - 0s 6ms/step - loss: 0.7372 - binary_accuracy: 0.2931 - val_loss: 0.7362 - val_binary_accuracy: 0.3015
Epoch 3/5
18/18 [==============================] - 0s 6ms/step - loss: 0.7371 - binary_accuracy: 0.2933 - val_loss: 0.7362 - val_binary_accuracy: 0.3015
Epoch 4/5
18/18 [==============================] - 0s 6ms/step - loss: 0.7371 - binary_accuracy: 0.2934 - val_loss: 0.7361 - val_binary_accuracy: 0.3015
Epoch 5/5
18/18 [==============================] - 0s 7ms/step - loss: 0.7370 - binary_accuracy: 0.2937 - val_loss: 0.7361 - val_binary_accuracy: 0.3022


In [25]:
model_ANN.save('./Models/model_ANN_trained.h5')

### Train XGBoost classifier

To not only test neural networks, we here test another standard machine learning algorithm. XGBoost has become a method of choice in recent Kaggle competitions (althought this does not mean that it will perform better than neural networks in the present case - "No Free Lunch" theorem):

In [32]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV

params = {
    'max_depth': [5, 7, 10]
}

gridsearch = GridSearchCV(estimator = xgb.XGBClassifier(
                          objective = "binary:logistic"
                          ),
                        param_grid = params,
                        scoring='accuracy',
                        n_jobs=1,
                        cv=5)

In [33]:
gridsearch.fit(x_train, y_train)

GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     gamma=None, gpu_id=None, grow_policy=None,
                                     importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None, max_bin=None,
                                     max_cat_to_onehot=None,
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
    

In [34]:
gridsearch.cv_results_, gridsearch.best_params_, gridsearch.best_score_

({'mean_fit_time': array([3.24635272, 4.41634746, 5.22779336]),
  'std_fit_time': array([0.32545536, 0.85535216, 0.15493848]),
  'mean_score_time': array([0.02274785, 0.02811713, 0.03941512]),
  'std_score_time': array([0.00126596, 0.00088646, 0.0050415 ]),
  'param_max_depth': masked_array(data=[5, 7, 10],
               mask=[False, False, False],
         fill_value='?',
              dtype=object),
  'params': [{'max_depth': 5}, {'max_depth': 7}, {'max_depth': 10}],
  'split0_test_score': array([0.78036972, 0.77985622, 0.77633509]),
  'split1_test_score': array([0.78572477, 0.78359742, 0.77882923]),
  'split2_test_score': array([0.79245837, 0.78857017, 0.78314137]),
  'split3_test_score': array([0.78409508, 0.7848287 , 0.77917981]),
  'split4_test_score': array([0.77932654, 0.77807938, 0.77653877]),
  'mean_test_score': array([0.78439489, 0.78298638, 0.77880485]),
  'std_test_score': array([0.00466408, 0.00371006, 0.00245688]),
  'rank_test_score': array([1, 2, 3], dtype=int32)},
 

In [35]:
from joblib import dump

dump(gridsearch, './Models/model_XGBoost.joblib') 

['./Models/model_XGBoost.joblib']